In [ ]:
import json

import igraph as ig
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import functions as sf
from pyspark import SparkConf
from pyspark.sql import SparkSession

import settings as s

In [ ]:
sns.set_theme(style="white", context="talk")

In [ ]:
config = [
    ("spark.driver.memory", "8g"),
    ("spark.worker.memory", "8g"),
]
spark = (
    SparkSession.builder.appName("testing")
    .config(conf=SparkConf().setAll(config))
    .getOrCreate()
)

In [ ]:
def update_source_target(input_data):
    return input_data.withColumn("source", sf.substring("source", 0, 8)).withColumn(
        "target", sf.substring("target", 0, 8)
    )

In [ ]:
%%time

data = update_source_target(spark.read.parquet(s.STAGED_DATA_LOCATION))
data_cases = update_source_target(
    spark.createDataFrame(pd.read_parquet(s.STAGED_CASES_DATA_LOCATION))
)
edges = data.select("source", "target").distinct().toPandas()
graph = ig.Graph.DataFrame(edges, use_vids=False, directed=True)

In [ ]:
is_ml = sf.col("is_laundering") == 1
ml_sources = data.where(is_ml).select("source").distinct().toPandas()["source"].tolist()
ml_targets = data.where(is_ml).select("target").distinct().toPandas()["target"].tolist()
ml_nodes = set(ml_sources).union(ml_targets)

ml_flow_edges = (
    data_cases.select("source", "target").toPandas().set_index(["source", "target"])
)
ml_flow_sources = data_cases.select("source").distinct().toPandas()["source"].tolist()
ml_flow_targets = data_cases.select("target").distinct().toPandas()["target"].tolist()
ml_flow_nodes = set(ml_flow_sources).union(ml_flow_targets)

per_overlap = len(ml_flow_nodes.intersection(ml_nodes)) / len(ml_nodes)
print(f"Percentage of ml-flow nodes in ml nodes: {round(per_overlap * 100, 2)}%")

In [ ]:
pd.DataFrame(ml_nodes, columns=["ml"]).to_parquet("ml_nodes.parquet")

In [ ]:
currency_rates = {
    "jpy": np.float32(0.009487665410827868),
    "cny": np.float32(0.14930721887033868),
    "cad": np.float32(0.7579775434031815),
    "sar": np.float32(0.2665884611958837),
    "aud": np.float32(0.7078143121927827),
    "ils": np.float32(0.29612081311363503),
    "chf": np.float32(1.0928961554056371),
    "usd": np.float32(1.0),
    "eur": np.float32(1.171783425225877),
    "rub": np.float32(0.012852809604990688),
    "gbp": np.float32(1.2916554735187644),
    "btc": np.float32(11879.132698717296),
    "inr": np.float32(0.013615817231245796),
    "mxn": np.float32(0.047296753463246695),
    "brl": np.float32(0.1771008654705292),
}

In [ ]:
flows = data_cases.toPandas()
flows.loc[:, "amount_usd"] = flows.apply(
    lambda x: x["source_amount"] * currency_rates[x["source_currency"]], axis=1
)
flows.to_parquet("flows.parquet")

In [ ]:
flows["id"].nunique()

In [ ]:
%%time

flow_stats = []
for flow_type, group in flows.groupby("type"):
    for flow, group_inner in group.groupby("id"):
        # Add turnover score
        left = (
            group_inner.loc[:, ["target", "amount_usd"]]
            .rename(columns={"target": "source"})
            .groupby("source")
            .agg({"amount_usd": "sum"})
        )
        right = group_inner.groupby("source").agg({"amount_usd": "sum"})
        result = left.join(right, how="outer", lsuffix="_left").fillna(0).reset_index()
        result.loc[:, "delta"] = result["amount_usd_left"] - result["amount_usd"]
        turnover = float(result[result["delta"] > 0]["delta"].sum())
        turnover = int(np.ceil(turnover)) or 1
        turnover_weight = (
            result.set_index("source").apply(
                lambda x: max([x["amount_usd_left"], x["amount_usd"]]), axis=1
            )
            / turnover
        ).to_dict()
        turnover_weight = {
            k: v / (sum(turnover_weight.values()) / len(turnover_weight)) for k, v in turnover_weight.items()
        }
        flow_data = {
            "id": flow,
            "type": flow_type,
            "turnover": turnover,
            "turnover_weight": json.dumps(turnover_weight),
        }
        flow_data["sub_type"] = group_inner.iloc[0]["sub_type"]
        group_inner = group_inner.sort_values("timestamp").reset_index(drop=True)
        dates = [x.to_pydatetime().date() for x in group_inner["timestamp"].tolist()]
        dates_shifted = dates[1:] + [dates[-1]]
        flow_data["total_days"] = (max(dates) - min(dates)).days + 1
        day_diffs = [(x - y).days + 1 for x, y in zip(dates_shifted, dates)]
        flow_data["max_days_diff"] = max(day_diffs)
        sources = set(group_inner["source"])
        targets = set(group_inner["target"])
        flow_data["number_transactions"] = group_inner["transaction_id"].nunique()
        flow_data["number_sources"] = len(sources)
        flow_data["number_targets"] = len(targets)
        flow_data["number_accounts"] = len(sources.union(targets))
        source_banks = set(group_inner["source_bank"])
        target_banks = set(group_inner["target_bank"])
        flow_data["number_source_banks"] = len(source_banks)
        flow_data["number_target_banks"] = len(target_banks)
        flow_data["number_banks"] = len(source_banks.union(target_banks))
        source_currencies = set(group_inner["source_currency"])
        target_currencies = set(group_inner["target_currency"])
        flow_data["number_source_currencies"] = len(source_currencies)
        flow_data["number_target_currencies"] = len(target_currencies)
        flow_data["number_currencies"] = len(source_currencies.union(target_currencies))
        sub_graph_induced = graph.induced_subgraph(sources.union(targets))
        sub_graph = ig.Graph.DataFrame(
            group_inner[["source", "target"]], use_vids=False, directed=True
        )
        distances = np.array(sub_graph.distances(mode="all"))
        max_distance = int(distances[distances < 1e6].max())
        flow_data["max_distance"] = max_distance
        flow_data["max_degree_in"] = max(sub_graph_induced.degree(mode="in"))
        flow_data["max_degree_out"] = max(sub_graph_induced.degree(mode="out"))
        flow_data["max_degree"] = max(sub_graph_induced.degree(mode="all"))
        flow_data["number_components"] = len(
            sub_graph.connected_components(mode="weak")
        )
        flow_stats.append(flow_data)
flow_stats = pd.DataFrame(flow_stats)
flow_stats.loc[:, "type"] = flow_stats.loc[:, "type"].str.lower()
flow_stats.loc[:, "sub_type"] = flow_stats.loc[:, "sub_type"].str.lower()
flow_stats.to_parquet("flow_stats.parquet")

In [ ]:
connected_filter = flow_stats["number_components"] == 1
flows_connected = flow_stats.loc[connected_filter, :].reset_index(drop=True)
flows_disconnected = flow_stats.loc[flow_stats["number_components"] > 1, :].reset_index(
    drop=True
)
print(
    f"Connected flows: {round((flows_connected.shape[0] / flow_stats.shape[0]) * 100, 2)}%"
)

In [ ]:
connected_counts = pd.DataFrame(
    flows_connected.groupby("type")["number_components"].count().sort_values()
).rename(columns={"number_components": "connected"})
disconnected_counts = pd.DataFrame(
    flows_disconnected.groupby("type")["number_components"].count().sort_values()
).rename(columns={"number_components": "disconnected"})
connected_vs_disconnected = connected_counts.join(
    disconnected_counts, how="outer"
).fillna(0)
connected_vs_disconnected.loc[:, "x"] = (
    connected_vs_disconnected["connected"] + connected_vs_disconnected["disconnected"]
)
connected_vs_disconnected = connected_vs_disconnected.sort_values("x", ascending=False)
del connected_vs_disconnected["x"]

connected_vs_disconnected.plot(kind="bar", stacked=True)
plt.gca().set_xlabel("Flow Type")
plt.gca().set_ylabel("Number of Flows")
plt.legend(loc="right", bbox_to_anchor=(1.5, 0.9))

In [ ]:
pd.DataFrame(
    flow_stats.groupby("type").agg(
        mean=("total_days", "mean"), maximum=("total_days", "max")
    )
).sort_values("maximum", ascending=False)

In [ ]:
pd.DataFrame(
    flow_stats.groupby("type").agg(
        mean=("total_days", "mean"), maximum=("total_days", "max")
    )
).sort_values("maximum").plot(kind="bar", stacked=False, legend=None)
plt.gca().set_xlabel("Flow Type")
plt.gca().set_ylabel("Max Number of Days")

In [ ]:
pd.DataFrame(flow_stats.groupby("type")["number_accounts"].max()).sort_values(
    "number_accounts"
).plot(kind="bar", stacked=False, legend=None)

In [ ]:
pd.DataFrame(
    flow_stats.groupby("type").agg(
        mean=("number_accounts", "mean"), maximum=("number_accounts", "max")
    )
).sort_values("maximum", ascending=False)

In [ ]:
pd.DataFrame(flow_stats.groupby("type")["number_banks"].mean()).sort_values(
    "number_banks"
).plot(kind="bar", stacked=False, legend=None)
plt.gca().set_xlabel("Flow Type")
plt.gca().set_ylabel("Average Number of Banks")

In [ ]:
pd.DataFrame(flow_stats.groupby("type")["max_distance"].max()).sort_values(
    "max_distance"
).plot(kind="bar", stacked=False, legend=None)
plt.gca().set_xlabel("Flow Type")
plt.gca().set_ylabel("Maximum Distance")

In [ ]:
pd.DataFrame(
    flow_stats.groupby("type").agg(
        mean=("max_degree", "mean"), minimum=("max_degree", "min")
    )
).sort_values("minimum", ascending=False)

In [ ]:
pd.DataFrame(flow_stats.groupby("type")["max_degree"].min()).sort_values(
    "max_degree"
).plot(kind="bar", stacked=False, legend=None)
plt.gca().set_xlabel("Flow Type")
plt.gca().set_ylabel("Minimum Degree")